In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)

train.fillna(0, inplace=True)

# Concatenate last two columns into a new column
train['Concatenated'] = train.apply(lambda x: ''.join(map(str, x[4:])), axis=1)
train['Concatenated'] = '[MESSAGE]' + train['text'] + '[CHANGE]' + train['Concatenated']
train.drop(columns = ['text'],inplace=True)
train = train.rename(columns={'3_labels':'label','Concatenated':'text'})
print(train['text'])


test = pd.read_csv('test.csv', index_col=0)

test.fillna(0, inplace=True)

test['Concatenated'] = test.apply(lambda x: ''.join(map(str, x[4:])), axis=1)
test['Concatenated'] = '[MESSAGE]' +  test['text'] +  '[CHANGE]' + test['Concatenated']
test.drop(columns = ['text'],inplace=True)
test = test.rename(columns={'3_labels':'label','Concatenated':'text'})


print(test['text'])



1173    [MESSAGE]Renames `StringLiteralAuditor` to `I1...
1273    [MESSAGE]Minor cleanup\n\n-ignore warning abou...
266     [MESSAGE]SVN: configure branches NPE--[CHANGE]...
98      [MESSAGE]HBASE-3387 Pair does not deep check a...
743     [MESSAGE]Changed syntax for "create class" com...
                              ...                        
945     [MESSAGE]fixed "hibernateManagedSession" mode ...
1381    [MESSAGE]DELTASPIKE-402 make DependentProvider...
1411    [MESSAGE]glib-2.0: add GLib.HashTable.foreach\...
1779    [MESSAGE]optimizations, export improvements\ng...
306     [MESSAGE]preferred focused component (IDEA-605...
Name: text, Length: 304, dtype: object
882     [MESSAGE]CAMEL-1078.  Fix potential NPE.--git-...
503     [MESSAGE][JBRULES-3263] fix jitted contraints ...
123     [MESSAGE]HBASE-11248-KeyOnlyKeyValue-toString(...
938     [MESSAGE]DataSourceUtils lets timeout exceptio...
1160    [MESSAGE]Removes unused dependencies/imports.\...
                              ...

In [3]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [4]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [5]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [6]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['commit_id', 'project', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_TYPE_CHAN

In [7]:
len(train)

304

In [8]:
train['label'].value_counts()

label
Perfective    102
Corrective    102
Adaptive      100
Name: count, dtype: int64

In [9]:
test['label'].value_counts()

label
Corrective    501
Perfective    498
Adaptive      490
Name: count, dtype: int64

In [10]:
len(train_dataset)

328

In [11]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [12]:
train_dataset

Dataset({
    features: ['commit_id', 'project', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_TYPE_CHAN

In [13]:
# train

In [22]:
model_id = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

In [23]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [24]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [25]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 69.61it/s]
***** Running training *****
  Num examples = 13120
  Num epochs = 1
  Total optimization steps = 820
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [02:31<00:00, 151.23s/it]
***** Running evaluation *****


CPU times: user 2min 2s, sys: 1min 2s, total: 3min 5s
Wall time: 2min 33s


{'accuracy': 0.7541974479516454,
 'recall': 0.7541974479516454,
 'f1': 0.7534746626318102}